In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import requests

In [ ]:
CHROMEDRIVER_PATH = "path/to/chromedriver"  # Update this path
INSTAGRAM_USERNAME = "your_username"
INSTAGRAM_PASSWORD = "your_password"

In [ ]:
def setup_driver():
    service = Service(CHROMEDRIVER_PATH)
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [ ]:
def login_instagram(driver):
    driver.get("https://www.instagram.com/")
    time.sleep(3)

In [ ]:
username_input = driver.find_element(By.NAME, "username")
    password_input = driver.find_element(By.NAME, "password")

    username_input.send_keys(INSTAGRAM_USERNAME)
    password_input.send_keys(INSTAGRAM_PASSWORD)
    password_input.send_keys(Keys.RETURN)

In [ ]:
    time.sleep(5)  # Wait for login to complete


In [ ]:
def get_image_urls(driver, hashtag, num_images=10):
    url = f"https://www.instagram.com/explore/tags/{hashtag}/"
    driver.get(url)
    time.sleep(5)

    image_urls = set()
    scroll_count = 0

    while len(image_urls) < num_images:
        images = driver.find_elements(By.TAG_NAME, "img")
        for img in images:
            src = img.get_attribute("src")
            if src and src not in image_urls:
                image_urls.add(src)
                if len(image_urls) >= num_images:
                    break

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        scroll_count += 1

        if scroll_count > 10:  # Stop after too many scrolls
            break

    return list(image_urls)

In [ ]:
def download_images(image_urls, folder="images"):
    os.makedirs(folder, exist_ok=True)
    
    for i, url in enumerate(image_urls):
        response = requests.get(url)
        if response.status_code == 200:
            with open(f"{folder}/image_{i+1}.jpg", "wb") as file:
                file.write(response.content)
            print(f"Downloaded {folder}/image_{i+1}.jpg")

# Main function
if __name__ == "__main__":
    driver = setup_driver()
    
    try:
        login_instagram(driver)
        cat_images = get_image_urls(driver, "cat", num_images=20)
        dog_images = get_image_urls(driver, "dog", num_images=20)

        print(f"Found {len(cat_images)} cat images and {len(dog_images)} dog images")

        download_images(cat_images, folder="cats")
        download_images(dog_images, folder="dogs")

    finally:
        driver.quit()